# Lambda³ NumPyro Framework - Google Colab Setup

This notebook sets up and tests the Lambda³ NumPyro framework in Google Colab.

## 1. Clone Repository and Install Dependencies

In [ ]:
# Clone the repository
!git clone https://github.com/miosync-masa/bayesian-event-detector.git
%cd bayesian-event-detector

In [ ]:
# Install required packages
!pip install numpy numba jax jaxlib numpyro arviz pandas scikit-learn matplotlib seaborn networkx yfinance

In [ ]:
# Add the lambda3_numpyro directory to Python path
import sys
sys.path.append('/content/bayesian-event-detector')

# Verify JAX installation and GPU availability
import jax
print(f"JAX version: {jax.__version__}")
print(f"JAX backend: {jax.default_backend()}")
print(f"GPU available: {jax.default_backend() == 'gpu'}")

## 2. Import Lambda³ Modules

In [ ]:
# Import Lambda³ modules
from lambda3_numpyro import (
    L3Config,
    extract_lambda3_features,
    analyze_pair,
    analyze_multiple_series,
    plot_features,
    PLOTTING_AVAILABLE
)

print(f"Lambda³ NumPyro successfully imported!")
print(f"Plotting available: {PLOTTING_AVAILABLE}")

## 3. Basic Feature Extraction Test

In [ ]:
# Generate synthetic data
import numpy as np

np.random.seed(42)
n_points = 500
t = np.linspace(0, 10*np.pi, n_points)

# Create two correlated series with some jumps
series_a = np.sin(t) + 0.1 * np.random.randn(n_points)
series_b = np.sin(t - 0.5) + 0.1 * np.random.randn(n_points)  # Lagged

# Add jumps
series_a[100] += 3
series_a[300] -= 3
series_b[105] += 3
series_b[305] -= 3

print(f"Generated {n_points} data points for two series")

In [ ]:
# Extract features
config = L3Config()
config.bayesian.draws = 1000  # Reduce for faster testing
config.bayesian.tune = 1000
config.bayesian.num_chains = 2

features_a = extract_lambda3_features(series_a, config, series_name='Series_A')
features_b = extract_lambda3_features(series_b, config, series_name='Series_B')

print(f"\nSeries A:")
print(f"  Positive jumps (ΔΛC⁺): {features_a.n_pos_jumps}")
print(f"  Negative jumps (ΔΛC⁻): {features_a.n_neg_jumps}")
print(f"  Mean tension (ρT): {features_a.mean_tension:.3f}")

print(f"\nSeries B:")
print(f"  Positive jumps (ΔΛC⁺): {features_b.n_pos_jumps}")
print(f"  Negative jumps (ΔΛC⁻): {features_b.n_neg_jumps}")
print(f"  Mean tension (ρT): {features_b.mean_tension:.3f}")

## 4. Visualize Features

In [ ]:
# Plot features
if PLOTTING_AVAILABLE:
    import matplotlib.pyplot as plt
    
    features_dict = {'Series_A': features_a, 'Series_B': features_b}
    plot_features(features_dict, title="Lambda³ Features")
    plt.show()

## 5. Pairwise Analysis Test

In [ ]:
# Run pairwise analysis
print("Running pairwise analysis...")
results = analyze_pair(
    'Series_A', 'Series_B',
    features_a, features_b,
    config,
    seed=42
)

print(f"\nAnalysis Results:")
print(f"  Max synchronization rate (σₛ): {results.sync_profile.max_sync_rate:.3f}")
print(f"  Optimal lag: {results.sync_profile.optimal_lag}")
print(f"\nInteraction effects:")
for effect, value in results.interaction_effects.items():
    if abs(value) > 0.01:
        print(f"  {effect}: {value:.3f}")

## 6. Test Financial Data Loading

In [ ]:
# Load financial data
from lambda3_numpyro import load_financial_data

# Download sample financial data
tickers = {
    "SPY": "SPY",
    "QQQ": "QQQ",
    "DIA": "DIA"
}

print("Downloading financial data...")
financial_data = load_financial_data(
    start_date="2023-01-01",
    end_date="2023-12-31",
    tickers=tickers,
    save_csv=False
)

print(f"\nDownloaded data for: {list(financial_data.keys())}")
for name, data in financial_data.items():
    print(f"  {name}: {len(data)} data points")

## 7. Multi-Series Analysis Test

In [ ]:
# Extract features from financial data
from lambda3_numpyro.feature import extract_features_dict

print("Extracting features from financial data...")
financial_features = extract_features_dict(financial_data, config)

# Run cross-analysis
print("\nRunning cross-analysis...")
cross_results = analyze_multiple_series(
    financial_features,
    config,
    show_progress=True
)

print(f"\nCross-Analysis Summary:")
print(f"  Series analyzed: {cross_results.n_series}")
print(f"  Pairs analyzed: {cross_results.n_pairs}")
print(f"  Network density: {cross_results.network_density:.1%}")

## 8. Visualize Results

In [ ]:
# Plot interaction matrix
if PLOTTING_AVAILABLE:
    from lambda3_numpyro.plot import plot_interaction_matrix
    
    series_names = cross_results.get_series_names()
    plot_interaction_matrix(
        cross_results.interaction_matrix,
        series_names
    )
    plt.show()

## 9. Run Unit Tests

In [ ]:
# Run tests using the command line interface
!cd /content/bayesian-event-detector && python -m lambda3_numpyro.main test --type unit --modules feature

## 10. Quick Performance Test

In [ ]:
# Test performance with different data sizes
import time

sizes = [100, 500, 1000, 5000]
times = []

for size in sizes:
    data = np.cumsum(np.random.randn(size))
    
    start = time.time()
    features = extract_lambda3_features(data, config)
    elapsed = time.time() - start
    
    times.append(elapsed)
    print(f"Size {size}: {elapsed:.3f} seconds")

# Plot performance
if PLOTTING_AVAILABLE:
    plt.figure(figsize=(8, 5))
    plt.plot(sizes, times, 'o-')
    plt.xlabel('Data Size')
    plt.ylabel('Time (seconds)')
    plt.title('Feature Extraction Performance')
    plt.grid(True)
    plt.show()